## 1. 什么是XGBoost

XGBoost 高效地实现了 GBDT 算法并进行了算法和工程上的许多改进。
XGBoost本质上还是一个GBDT，但是力争把速度和效率发挥到极致，所以叫X (Extreme) GBoosted。包括前面说过，两者都是boosting方法。

关于GBDT，可以查看前一篇的介绍，[点此跳转](https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/Gradient%20Boosting%20Decision%20Tree.ipynb)。

### 1.1 XGBoost树的定义

先来举个**例子**，我们要预测一家人对电子游戏的喜好程度，考虑到年轻和年老相比，年轻更可能喜欢电子游戏，以及男性和女性相比，男性更喜欢电子游戏，故先根据年龄大小区分小孩和大人，然后再通过性别区分开是男是女，逐一给各人在电子游戏喜好程度上打分，如下图所示。
<img src="https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/20.png?raw=true" width="500"/>

就这样，训练出了 2 棵树 tree1 和 tree2，类似之前 gbdt 的原理，两棵树的结论累加起来便是最终的结论，所以小孩的预测分数就是两棵树中小孩所落到的结点的分数相加：2 + 0.9 = 2.9。爷爷的预测分数同理：-1 + （-0.9）= -1.9。具体如下图所示：
<img src="https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/21.png?raw=true" width="520"/>

恩？这不是跟上文介绍的 GBDT 异曲同工么？

事实上，如果不考虑工程实现、解决问题上的一些差异，XGBoost 与 GBDT **比较大的不同就是目标函数的定义**。XGBoost 的目标函数如下图所示：
<img src="https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/22.png?raw=true" width="500"/>

其中：

- 红色箭头所指向的L 即为损失函数（比如平方损失函数：$l(y_i,y^i)=(y_i-y^i)^2$
- 红色方框所框起来的是正则项（包括 L1 正则、L2 正则）
- 红色圆圈所圈起来的为常数项
- 对于 f(x)，XGBoost 利用泰勒展开三项，做一个近似。**f(x)表示的是其中一颗回归树。**

看到这里可能有些读者会头晕了，这么多公式，**我在这里只做一个简要式的讲解，具体的算法细节和公式求解请查看这篇博文，讲得很仔细**：[通俗理解kaggle比赛大杀器xgboost](https://blog.csdn.net/v_JULY_v/article/details/81410574)

XGBoost 的**核心算法思想**不难，基本就是：

1. 不断地添加树，不断地进行特征分裂来生长一棵树，每次添加一个树，其实是学习一个新函数 **f(x)**，去拟合上次预测的残差。
2. 当我们训练完成得到 k 棵树，我们要预测一个样本的分数，其实就是根据这个样本的特征，在每棵树中会落到对应的一个叶子节点，每个叶子节点就对应一个分数
3. 最后只需要将每棵树对应的分数加起来就是该样本的预测值。

显然，我们的目标是要使得树群的预测值 $y_i^{'}$ 尽量接近真实值 $y_i$，而且有尽量大的泛化能力。类似之前 GBDT 的套路，XGBoost 也是需要将多棵树的得分累加得到最终的预测得分（每一次迭代，都在现有树的基础上，增加一棵树去拟合前面树的预测结果与真实值之间的残差）。
<img src="https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/23.png?raw=true" width="500"/>

那接下来，我们如何选择每一轮加入什么 f 呢？答案是非常直接的，选取一个 f 来使得我们的目标函数尽量最大地降低。这里 f 可以使用泰勒展开公式近似。
<img src="https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/24.png?raw=true" width="500"/>

实质是把样本分配到叶子结点会对应一个 obj，优化过程就是 obj 优化。也就是分裂节点到叶子不同的组合，不同的组合对应不同 obj，所有的优化围绕这个思想展开。到目前为止我们讨论了目标函数中的第一个部分：训练误差。接下来我们讨论目标函数的第二个部分：正则项，即如何定义树的复杂度。

### 1.2 正则项：树的复杂度

XGBoost对树的复杂度包含了两个部分：

- 一个是树里面叶子节点的个数 T
- 一个是树上叶子节点的得分 w 的 L2 模平方（对 w 进行 L2 正则化，相当于针对每个叶结点的得分增加 L2 平滑，目的是为了避免过拟合）
<img src="https://github.com/yunjcai/ML-DL-Training-Materials/blob/main/03%20Decision%20Tree/image/25.png?raw=true" width="500"/>

我们再来看一下 XGBoost 的目标函数（损失函数揭示训练误差 + 正则化定义复杂度）：

$L(\phi)=\sum_{i}l(y_i^{'}-y_i)+\sum_k\Omega(f_t)$

正则化公式也就是目标函数的后半部分，对于上式而言，$y_i^{'}$ 是整个累加模型的输出，正则化项 $∑kΩ(ft)$ 是则表示树的复杂度的函数，值越小复杂度越低，泛化能力越强。

### 1.3 树该怎么长

很有意思的一个事是，我们从头到尾了解了 xgboost 如何优化、如何计算，但树到底长啥样，我们却一直没看到。很显然，一棵树的生成是由一个节点一分为二，然后不断分裂最终形成为整棵树。那么树怎么分裂的就成为了接下来我们要探讨的关键。对于一个叶子节点如何进行分裂，XGBoost 作者在其原始论文中给出了一种分裂节点的方法：**枚举所有不同树结构的贪心法**

不断地枚举不同树的结构，然后利用打分函数来寻找出一个最优结构的树，接着加入到模型中，不断重复这样的操作。这个寻找的过程使用的就是**贪心算法**。选择一个 feature 分裂，计算 loss function 最小值，然后再选一个 feature 分裂，又得到一个 loss function 最小值，你枚举完，找一个效果最好的，把树给分裂，就得到了小树苗。

总而言之，XGBoost 使用了和CART回归树一样的想法，利用贪婪算法，遍历所有特征的所有特征划分点，不同的是使用的目标函数不一样。具体做法就是分裂后的目标函数值比单子叶子节点的目标函数的增益，同时为了限制树生长过深，还加了个阈值，只有当增益大于该阈值才进行分裂。从而继续分裂，形成一棵树，再形成一棵树，**每次在上一次的预测基础上取最优进一步分裂/建树。**

### 1.4 如何停止树的循环生成

凡是这种循环迭代的方式必定有停止条件，什么时候停止呢？简言之，设置树的最大深度、当样本权重和小于设定阈值时停止生长以防止过拟合。具体而言，则

1. 当引入的分裂带来的增益小于设定阀值的时候，我们可以忽略掉这个分裂，所以并不是每一次分裂 loss function 整体都会增加的，有点预剪枝的意思，阈值参数为（即正则项里叶子节点数 T 的系数）；
2. 当树达到最大深度时则停止建立决策树，设置一个超参数 max_depth，避免树太深导致学习局部样本，从而过拟合；
3. 样本权重和小于设定阈值时则停止建树。什么意思呢，即涉及到一个超参数-最小的样本权重和 min_child_weight，和 GBM 的 min_child_leaf 参数类似，但不完全一样。大意就是一个叶子节点样本太少了，也终止同样是防止过拟合；

## 2. XGBoost 与GBDT 有什么不同

除了算法上与传统的 GBDT 有一些不同外，XGBoost 还在工程实现上做了大量的优化。总的来说，两者之间的区别和联系可以总结成以下几个方面。 

1. GBDT 是机器学习算法，XGBoost 是该算法的工程实现。
2. 在使用 CART 作为基分类器时，XGBoost 显式地加入了正则项来控制模 型的复杂度，有利于防止过拟合，从而提高模型的泛化能力。
3. GBDT 在模型训练时只使用了代价函数的一阶导数信息，XGBoost 对代 价函数进行二阶泰勒展开，可以同时使用一阶和二阶导数。
4. 传统的 GBDT 采用 CART 作为基分类器，XGBoost 支持多种类型的基分类器，比如线性分类器。
5. 传统的 GBDT 在每轮迭代时使用全部的数据，XGBoost 则采用了与随机森林相似的策略，支持对数据进行采样。
6. 传统的 GBDT 没有设计对缺失值进行处理，XGBoost 能够自动学习出缺失值的处理策略。

## 3. 为什么 XGBoost 要用泰勒展开，优势在哪里？

XGBoost 使用了一阶和二阶偏导, 二阶导数有利于梯度下降的更快更准. 使用泰勒展开取得函数做自变量的二阶导数形式, 可以在不选定损失函数具体形式的情况下, 仅仅依靠输入数据的值就可以进行叶子分裂优化计算, 本质上也就把损失函数的选取和模型算法优化/参数选择分开了. 这种去耦合增加了 XGBoost 的适用性, 使得它按需选取损失函数, 可以用于分类, 也可以用于回归。

## 4. [代码实现]()

## 5. XGBoost 参数调节

XGBoost 的参数主要分为三种：

General Parameters: 控制总体的功能
Booster Parameters: 控制单个学习器的属性
Learning Task Parameters: 控制调优的步骤

#### General Parameters:
* booster [default = gbtree] 选择每一次迭代中，模型的种类. 有两个选择:
 * gbtree: 基于树的模型
 * gblinear: 线性模型
* silent [default = 0]:
 * 设为 1 则不打印执行信息。设为 0 打印信息
* nthread [default to maximum number of threads available if not set]
 * 这个是设置并发执行的信息，设置在几个核上并发
 * 如果你希望使用 CPU 全部的核，那就不要输入这个参数，算法会自动检测它。

#### Booster Parameters：
有2种 booster，tree booster 的表现远远胜过 linear booster，所以 linear booster 很少用到。

* eta [default = 0.3]
 * 类似于 GBM 里面的 learning rate
 * 通过在每一步中缩小权重来让模型更加鲁棒
 * 一般常用的数值: 0.01-0.2
 
* min_child_weight [default = 1]
 * Defines the minimum sum of weights of all observations required in a child.
 * Used to control over-fitting. Higher values prevent a model from learning relations which might be highly specific to the particular sample selected for a tree. 这个参数用于避免过拟合。当它的值较大时，可以避免模型学习到局部的特殊样本。
 * 但是如果这个值过高，会导致欠拟合。这个参数需要使用CV来调整。
 
* max_depth [default = 6]
 * The maximum depth of a tree, same as GBM. 设置树的最大深度
 * 控制子树中样本数占总的样本数的最低比例
 * Used to control over-fitting as higher depth will allow model to learn relations very specific to a particular sample. 控制过拟合，如果树的深度太大会导致过拟合
 * Should be tuned using CV. 应该使用CV来调节。
 * Typical values: 3-10
 
* max_leaf_nodes
 * The maximum number of terminal nodes or leaves in a tree.
 * 叶子节点的最大值
 * Can be defined in place of max_depth. Since binary trees are created, a depth of ‘n’ would produce a maximum of 2^n leaves.
 * 也是为了通过树的规模来控制过拟合
 * If this is defined, GBM will ignore max_depth. 如果叶子树确定了，对于2叉树来说高度也就定了，此时以叶子树确定的高度为准
 
* gamma [default = 0]
 * A node is split only when the resulting split gives a positive reduction in the loss function. Gamma specifies the minimum loss reduction required to make a split. 如果分裂能够使 loss 函数减小的值大于 gamma，则这个节点才分裂。gamma 设置了这个减小的最低阈值。如果 gamma 设置为0，表示只要使得 loss 函数减少，就分裂
 * Makes the algorithm conservative. The values can vary depending on the loss function and should be tuned. 这个值会跟具体的 loss 函数相关，需要调节
* max_delta_step [default = 0]
 * In maximum delta step we allow each tree’s weight estimation to be. If the value is set to 0, it means there is no constraint. If it is set to a positive value, it can help making the update step more conservative. 如果参数设置为 0，表示没有限制。如果设置为一个正值，会使得更新步更加谨慎。（关于这个参数我还是没有完全理解透彻。。。）
Usually this parameter is not needed, but it might help in logistic regression when class is extremely imbalanced.
不是很经常用，但是在逻辑回归时候，使用它可以处理类别不平衡问题。
subsample [default=1]
Same as the subsample of GBM. Denotes the fraction of observations to be randomly samples for each tree.
对原数据集进行随机采样来构建单个树。这个参数代表了在构建树时候 对原数据集采样的百分比。eg：如果设为0.8表示随机抽取样本中80%的个体来构建树。
Lower values make the algorithm more conservative and prevents overfitting but too small values might lead to under-fitting.
相对小点的数值可以防止过拟合，但是过小的数值会导致欠拟合（因为采样过小）。
Typical values: 0.5-1
一般取值 0.5 到 1
colsample_bytree [default=1]
Similar to max_features in GBM. Denotes the fraction of columns to be randomly samples for each tree.
创建树的时候，从所有的列中选取的比例。e.g：如果设为0.8表示随机抽取80%的列 用来创建树
Typical values: 0.5-1
colsample_bylevel [default=1]
Denotes the subsample ratio of columns for each split, in each level.
I don’t use this often because subsample and colsample_bytree will do the job for you. but you can explore further if you feel so.
lambda [default=1]
L2 regularization term on weights (analogous to Ridge regression)
L2正则项，类似于Ridge Regression
This used to handle the regularization part of XGBoost. Though many data scientists don’t use it often, it should be explored to reduce overfitting.
可以用来考虑降低过拟合，L2本身可以防止过分看重某个特定的特征。尽量考虑尽量多的特征纳入模型。
alpha [default=0]
L1 regularization term on weight (analogous to Lasso regression)
L1正则。 类似于lasso
Can be used in case of very high dimensionality so that the algorithm runs faster when implemented
L1正则有助于产生稀疏的数据，这样有助于提升计算的速度
scale_pos_weight [default=1]
A value greater than 0 should be used in case of high class imbalance as it helps in faster convergence.